Since you want to get know my thought process, I'm going 'think out loud' a bit with my comments in this notebook. 

In [2]:
import pandas as pd
import numpy as np
import os

# Stage 1: get to know the data

In [3]:
# let's get all our data together to better understand overall distributions
os.listdir('data') # unclear why archive contains the same filenames; going to shelve that investigation for now
filenames = os.listdir('data')
filenames.remove('archive')
filenames

['2016Q1.csv.gz',
 '2016Q2.csv.gz',
 '2016Q3.csv.gz',
 '2016Q4.csv.gz',
 '2017Q1.csv.gz',
 '2017Q2.csv.gz',
 '2017Q3.csv.gz',
 '2017Q4.csv.gz']

In [27]:
# concat together - retain the year and quarter in case that will be relevant
dat_list = []
for datfile in filenames:
    holding_df = pd.read_csv('data/' + datfile, nrows=100, compression='gzip')
    holding_df['year'] = datfile[0:4]
    holding_df['quarter'] = datfile[4:6]
    dat_list.append(holding_df)
loans_df = pd.concat(dat_list)
loans_df.head()

,id,funded_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,loan_status,...,percent_bc_gt_75,pub_rec_bankruptcies,total_bal_ex_mort,total_bc_limit,revol_bal_joint,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,year,quarter
0,73688651,4000,36 months,11.99%,132.84,Fraud analyst,8 years,RENT,45000.0,Fully Paid,...,0.0,0,1102,7500,NaN,NaN,NaN,NaN,2016,Q1
1,75678977,7200,36 months,11.47%,237.33,Client Relations Manager,< 1 year,RENT,85000.0,Current,...,100.0,0,33021,14000,NaN,NaN,NaN,NaN,2016,Q1
2,71905793,20000,36 months,16.29%,706.01,Registerd nurse,8 years,MORTGAGE,56000.0,Current,...,75.0,2,54298,9700,NaN,NaN,NaN,NaN,2016,Q1
3,74231221,16000,60 months,12.99%,363.97,Financial Advisor,5 years,MORTGAGE,110000.0,Current,...,87.5,0,45733,39400,NaN,NaN,NaN,NaN,2016,Q1
4,68884968,28000,60 months,15.31%,670.69,Pharmacist-in-Charge,3 years,MORTGAGE,180000.0,Current,...,100.0,0,273107,51800,NaN,NaN,NaN,NaN,2016,Q1


In [5]:
# is our id globally unique?
print(loans_df.shape)
len(loans_df.id.unique())
# appears that we have different loans in every time period
# I don't fully understand how that would come to pass sequentially but that would be a domain knowledge thing to understand in more time :)

(800, 57)


800

In [6]:
# understand our outcome of interest
loans_df["loan_status"].value_counts()
# had to look up charged off - this looks bad for an investor! Late is of course also a bad sign. 
# both current and fully paid are on the positive side, but together that makes our dataset quite imbalanced
# most loans are current, and only ~14.4% are in some way delinquent. 

Current               456
Fully Paid            229
Charged Off            84
Late (31-120 days)     19
In Grace Period         9
Late (16-30 days)       3
Name: loan_status, dtype: int64

In [34]:
# check balance over time
loans_df.groupby("year")["loan_status"].value_counts(normalize=True)

year  loan_status       
2016  Current               0.4750
      Fully Paid            0.3650
      Charged Off           0.1275
      Late (31-120 days)    0.0175
      In Grace Period       0.0100
      Late (16-30 days)     0.0050
2017  Current               0.6650
      Fully Paid            0.2075
      Charged Off           0.0825
      Late (31-120 days)    0.0300
      In Grace Period       0.0125
      Late (16-30 days)     0.0025
Name: loan_status, dtype: float64

In [35]:
loans_df.groupby("quarter")["loan_status"].value_counts(normalize=True)

quarter  loan_status       
Q1       Current               0.585
         Fully Paid            0.295
         Charged Off           0.100
         Late (31-120 days)    0.020
Q2       Current               0.495
         Fully Paid            0.295
         Charged Off           0.145
         Late (31-120 days)    0.045
         In Grace Period       0.015
         Late (16-30 days)     0.005
Q3       Current               0.570
         Fully Paid            0.290
         Charged Off           0.115
         In Grace Period       0.010
         Late (31-120 days)    0.010
         Late (16-30 days)     0.005
Q4       Current               0.630
         Fully Paid            0.265
         Charged Off           0.060
         In Grace Period       0.020
         Late (31-120 days)    0.020
         Late (16-30 days)     0.005
Name: loan_status, dtype: float64

In [ ]:
# there is some variation - thinking I will leave quarter in as a feature

## Problem framing
I can imagine a couple different ways to approach this. We could focus on being able to predict loans that will be fully paid, and construct a binary outcome accordingly. However, thinking in terms of which loans are best to invest in, we will also want to consider things like the term and interest rate. If we can be confident that some larger set of loans are very likely to stay current (and eventually be paid off), representing roughly equally low risk, we will be able to further optimize our choice based on which will provide the highest return. Therefore, I'm still going to construct a binary, just with both current and fully paid as the 1 - besides my logic above, I don't think it's useful information for our classifier to differentiate between each negative outcome category. There are so few 'bad' outcomes that we might as well avoid them altogether, even if, say, sometimes the grace period loans bounce back. 

In [53]:
# construct outcome
loans_df["good_loan_status_binary"] = np.where(loans_df.loan_status.isin(['Current', 'Fully Paid']), 1, 0)
loans_df["good_loan_status_binary"].value_counts()

1    685
0    115
Name: good_loan_status_binary, dtype: int64

# Stage 2: pre-process features
Normally I would spend a lot longer on this step, and test out a few different approaches in a pipeline. We have a fairly high number of features here. I may test for highly correlated features, or test out an algorithmic attempt at dimensionality reduction. I may evaluate whether some numeric features should be binned. I may more closely evaluate the distributions of my numeric features and apply some statistical transformations. With categorical variables like emp_title with many distinct values, I may seek to collapse the values into fewer categories. Since this is just an exercise, I'm just going to do a simple process for getting a baseline performance, deal with NAs and scaling numerics, encode categoricals and move on. 


In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [28]:
# assess missingness
loans_df.isna().sum()

id                                  0
funded_amnt                         0
term                                0
int_rate                            0
installment                         0
emp_title                          47
emp_length                         47
home_ownership                      0
annual_inc                          0
loan_status                         0
purpose                             0
zip_code                            0
addr_state                          0
dti                                 0
delinq_2yrs                         0
earliest_cr_line                    0
fico_range_high                     0
inq_last_6mths                      0
mths_since_last_delinq            385
open_acc                            0
revol_bal                           0
revol_util                          1
total_acc                           0
application_type                    0
annual_inc_joint                  756
dti_joint                         756
open_act_il 

In [29]:
# I'm going to cut the sec_ and _joint columns because they are almost all missing and appear unlikely to helpful.
# The mths_ columns have a lot missing but seem very relevant - I'll create missing flags for them
loans_df["mths_since_last_delinq_missing"] = np.where(loans_df["mths_since_last_delinq"].isna(), 1, 0)
loans_df["mths_since_recent_bc_missing"] = np.where(loans_df["mths_since_recent_bc"].isna(), 1, 0)
loans_df["mths_since_recent_bc_dlq_missing"] = np.where(loans_df["mths_since_recent_bc_dlq"].isna(), 1, 0)
loans_df["mths_since_recent_revol_delinq_missing"] = np.where(loans_df["mths_since_recent_revol_delinq"].isna(), 1, 0)
# I'll also impute with 0; I don't want to use the mean and anything more sophisticated I'm calling out of scope for this exercise
loans_df = loans_df.apply(lambda x: x.fillna(0) if 'mths_' in x.name else x, axis=0) 
# I'll fil emp_ fields with "unknown", strip whitespace while we're at it
loans_df = loans_df.apply(lambda x: x.str.strip().fillna("unknown") if 'emp_' in x.name else x, axis=0) 
# the _util  and percent_ columns are a bit harder for me to intuit potential relevancy; going to impute the min, 0, for now
loans_df = loans_df.apply(lambda x: x.fillna(0) if '_util' in x.name else x, axis=0) 
loans_df["percent_bc_gt_75"] = loans_df["percent_bc_gt_75"].fillna(0)

# finally, drop the ones with missing I don't want
loans_df = loans_df.dropna(axis = 1)
loans_df.shape

(800, 54)

In [30]:
# verify
loans_df.isna().sum()

id                                        0
funded_amnt                               0
term                                      0
int_rate                                  0
installment                               0
emp_title                                 0
emp_length                                0
home_ownership                            0
annual_inc                                0
loan_status                               0
purpose                                   0
zip_code                                  0
addr_state                                0
dti                                       0
delinq_2yrs                               0
earliest_cr_line                          0
fico_range_high                           0
inq_last_6mths                            0
mths_since_last_delinq                    0
open_acc                                  0
revol_bal                                 0
revol_util                                0
total_acc                       

In [58]:
# ad-hoc cleaning: remove percent symbols so interest rate can be numeric
loans_df.int_rate = loans_df.int_rate.str.rstrip("%").astype(float)

In [59]:
# get to know my numeric features
loans_df.describe()

,id,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_high,inq_last_6mths,mths_since_last_delinq,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,total_bal_ex_mort,total_bc_limit,mths_since_last_delinq_missing,mths_since_recent_bc_missing,mths_since_recent_bc_dlq_missing,mths_since_recent_revol_delinq_missing,good_loan_status_binary
count,8.000000e+02,800.000000,800.000000,800.000000,8.000000e+02,800.000000,800.000000,800.000000,800.000000,800.000000,...,800.000000,800.000000,800.000000,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,9.713469e+07,15034.062500,13.072437,453.275988,8.591401e+04,19.072225,0.307500,701.957500,0.506250,18.078750,...,93.745000,40.581250,0.133750,51865.631250,23150.67500,0.481250,0.012500,0.736250,0.631250,0.856250
std,1.684950e+07,9436.268377,4.839222,284.445941,1.606334e+05,13.215941,0.863931,32.772943,0.834165,23.241563,...,9.101012,36.228071,0.372202,50476.990497,22253.88379,0.499961,0.111172,0.440941,0.482768,0.351056
min,6.800013e+07,1000.000000,5.320000,32.010000,3.000000e+03,0.110000,0.000000,664.000000,0.000000,0.000000,...,30.800000,0.000000,0.000000,75.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.368949e+07,7500.000000,9.490000,240.920000,4.714100e+04,12.167500,0.000000,679.000000,0.000000,0.000000,...,90.375000,0.000000,0.000000,21755.250000,9000.00000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,9.533599e+07,12312.500000,12.620000,371.640000,6.700000e+04,18.240000,0.000000,694.000000,0.000000,4.000000,...,97.600000,33.300000,0.000000,37574.500000,16800.00000,0.000000,0.000000,1.000000,1.000000,1.000000
75%,1.118538e+08,20000.000000,15.990000,618.422500,9.500000e+04,24.560000,0.000000,719.000000,1.000000,33.000000,...,100.000000,66.700000,0.000000,64311.750000,29925.00000,1.000000,0.000000,1.000000,1.000000,1.000000
max,1.263496e+08,40000.000000,30.990000,1415.780000,4.266080e+06,272.000000,9.000000,850.000000,5.000000,85.000000,...,100.000000,100.000000,3.000000,494068.000000,184020.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
cat_features = ['emp_title', 'emp_length', 'term', 'home_ownership', 'purpose', 'addr_state', 'earliest_cr_line', 'application_type', 'quarter']
# to do: make earliest_cr_line ordinal by casting to dates, and maybe emp_length by doing something about the unknown category
num_features = list(loans_df.select_dtypes('number'))
num_features.remove('id')
num_features.remove('good_loan_status_binary')

In [78]:
# set up pre-processing pipelines

# Encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
# Scaler
std_scaler = StandardScaler()


basic_ohe_transformer = Pipeline(steps=[
    ('onehot', ohe)])

zero_scale_transformer = Pipeline(steps=[
    ('scaler', std_scaler)])

# will drop columns we didn't include in our list
simple_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', basic_ohe_transformer, cat_features),
        ('num', zero_scale_transformer, num_features)]
        )

# Stage 3: Model, evaluate

In [38]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score, precision_score, average_precision_score, precision_recall_curve, roc_curve

In [119]:
# split data into train and test sets
# remove outcome explicitly from X
loans_df = loans_df.reset_index(drop=True)
X = loans_df.drop(columns=['good_loan_status_binary', 'loan_status'])
y = loans_df.good_loan_status_binary
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [80]:
# run some baseline models
lgr = LogisticRegression(solver = 'liblinear',max_iter=1000)
rf = RandomForestClassifier(n_jobs=4)

In [140]:
clf = Pipeline(steps=[('preprocessor', simple_preprocessor),
                      ('classifier', lgr)])

clf.fit(X_train,y_train)
clf.score(X_test, y_test)

0.8446969696969697

In [147]:
clf = Pipeline(steps=[('preprocessor', simple_preprocessor),
                      ('classifier', rf)]) 

clf.fit(X_train,y_train)
clf.score(X_test, y_test)

0.875

In [148]:
# we're out of the gate better than random for accuracy, which is something..
# What if we just want the top, say, 10% of probabilities of success to choose our investments from?
# probability to be good
raw_risks = clf.predict_proba(X_test)[:,1]
test_risk_df = pd.DataFrame(index=X_test.index,
                            data=raw_risks,
                            columns=["succes_prob"])
test_risk_df['label'] = y_test
test_risk_df
test_risk_df.sort_values('succes_prob', inplace=True, ascending=False)
top_k_pct = int(np.ceil(0.1 * test_risk_df.shape[0]))
predictions = test_risk_df.head(top_k_pct)
precision = predictions['label'].mean()
precision

1.0

In [128]:
# well that is great. let's make a function to easy try different values of k
def prec_at_k(clf, X, y, k=0.1):
    raw_risks = clf.predict_proba(X_test)[:,1]
    test_risk_df = pd.DataFrame(index=X_test.index,
                                data=raw_risks,
                                columns=["succes_prob"])
    test_risk_df['label'] = y_test
    test_risk_df
    test_risk_df.sort_values('succes_prob', inplace=True, ascending=False)
    top_k_pct = int(np.ceil(k * test_risk_df.shape[0]))
    predictions = test_risk_df.head(top_k_pct)
    precision = predictions['label'].mean()
    precision
    
    return precision

In [152]:
prec_at_k(clf, X_test, y_test, k=0.10)

1.0

In [153]:
prec_at_k(clf, X_test, y_test, k=0.20)

0.9811320754716981

In [154]:
prec_at_k(clf, X_test, y_test, k=0.30)

0.9625

# Stage 4: Conclusions
A lot of iteration and fine-tuning is left out here. However, given that our straightforward model is able to identify the top 10% of loans likely to stay current apparently perfectly, I think we've got a pretty strong basis to start making recommendations. From those, let's look at terms and interest rates.

In [137]:
test_risk_df.shape

(264, 2)

In [139]:
#make sure this works as expected
test_risk_df.join(loans_df).shape

(264, 57)

In [136]:
# join on index
candidates = test_risk_df.join(loans_df).head(27)
candidates.sort_values("int_rate", ascending=False)
# it looks like will still have sime higher rate, shorter term options to choose from
# when we apply this model to new loans! (from above, rates above 16% are in the top quartile)

,succes_prob,label,id,funded_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,...,pub_rec_bankruptcies,total_bal_ex_mort,total_bc_limit,year,quarter,mths_since_last_delinq_missing,mths_since_recent_bc_missing,mths_since_recent_bc_dlq_missing,mths_since_recent_revol_delinq_missing,good_loan_status_binary
544,0.96,1,106997566,17150,60 months,18.99,444.79,KITCHEN MANAGER,10+ years,MORTGAGE,...,1,18452,20400,2017,Q2,0,0,1,0,1
586,0.96,1,111674113,10000,60 months,18.06,254.27,Maintenance Technician,3 years,MORTGAGE,...,0,24618,15300,2017,Q2,1,0,1,1,1
305,0.95,1,94727073,35000,36 months,17.99,1265.16,Truck driver,10+ years,MORTGAGE,...,0,43076,64400,2016,Q4,1,0,1,1,1
685,0.97,1,116266243,12000,36 months,16.02,422.01,Office manager,10+ years,RENT,...,0,14212,6900,2017,Q3,0,0,1,0,1
289,0.97,1,89068343,16300,60 months,14.49,383.43,Agent,10+ years,MORTGAGE,...,0,44204,17200,2016,Q3,1,0,1,1,1
696,0.97,1,112501736,15000,60 months,14.08,349.65,Business Systems Analyst,2 years,MORTGAGE,...,0,29896,13500,2017,Q3,0,0,0,0,1
426,0.99,1,102434781,18000,60 months,13.99,418.74,truck driver,10+ years,MORTGAGE,...,0,15516,5300,2017,Q1,0,0,1,1,1
97,0.97,1,73915364,4000,36 months,13.67,136.08,unknown,unknown,RENT,...,0,3642,0,2016,Q1,1,1,1,1,1
350,0.99,1,94276765,16000,36 months,11.44,527.16,Customer service rep,9 years,MORTGAGE,...,1,9423,6600,2016,Q4,1,0,1,1,1
441,0.97,1,98745862,12000,36 months,11.39,395.09,Software Developer,10+ years,RENT,...,0,19348,9000,2017,Q1,0,0,0,0,1
